In [1]:
import os
import shutil
from pathlib import Path
from ipywidgets import interact

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from core.Case import *
from core.globals import *
from core.Log import *


from methods.plots import *
from methods.preprocessing import *
from methods.intensity import *
from methods.slicing import *


C:\Users\sulei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
'''
	find out at what point the orientation changes from LAS to RAS
	what vol.data.transpose(2,1,0) does to the shape & orientation
'''
#print(f"  :  {}")

caseID = "CNTRL_BJB_42898_78F"
case = Case(caseID)
vol = case.croppedCT
linear=F
spacing=[1.0]*3
shape=(64,64,64)
affine = vol.affine

#print(f"vol.shape: {vol.shape}, \norigin: {vol.origin}, \nspacing: {vol.spacing}")


In [27]:

img = sitk.GetImageFromArray(vol.data.transpose(2,1,0))
#print(f"cropped.shape XYZ: {croopedshape}, \norigin: {vol.origin}, \nspacing: {vol.spacing}")
#print(f"img.GetSize():  {img.GetSize()} - img shape matches CT")
#print(f"img.GetDirection(): {img.GetDirection()}, img.GetOrigin(): {img.GetOrigin()}, img.GetSpacing(): {img.GetSpacing()}")
#print(f" ct affine :  {affine}")
'''
img.GetDirection() is RAS, not LAS.
X: [i, _, _] -->  [+1, 0, 0] --> R ---> [-1, 0, 0] ---> L
Y: [_, i, _] -->  [0, +1, 0] --> A
Z: [_, _, i] -->  [0, 0, +1] --> S
'''
print(f"vol.spacing: {vol.spacing}")
#print(tuple(np.sqrt((vol.affine[:3, :3]**2).sum(axis=0))))
img = sitk.GetImageFromArray(vol.data.transpose(2,1,0))
img.SetDirection(tuple(vol.affine[:3, :3].flatten()))   				# Set orientation
img.SetOrigin(tuple(vol.affine[:3, 3]))                  				# Set origin
img.SetSpacing(tuple(np.sqrt((vol.affine[:3, :3]**2).sum(axis=0))))
print(f"img.GetDirection(): {img.GetDirection()}, \nimg.GetOrigin(): {img.GetOrigin()}, \nimg.GetSpacing(): {img.GetSpacing()}")


vol.spacing: (0.68, 0.68, 2.0)
img.GetDirection(): (-0.6800000071525574, 0.0, 0.0, 0.0, 0.6800000071525574, 0.0, 0.0, 0.0, 2.0), 
img.GetOrigin(): (169.97219848632812, -173.60159301757812, 1514.5), 
img.GetSpacing(): (0.6800000071525574, 0.6800000071525574, 2.0)


In [ ]:
img = sitk.GetImageFromArray(vol.data.transpose(2,1,0))
img.SetDirection(tuple(vol.affine[:3, :3].flatten()))   				# Set orientation
img.SetOrigin(tuple(vol.affine[:3, 3]))                  				# Set origin
img.SetSpacing(tuple(np.sqrt((vol.affine[:3, :3]**2).sum(axis=0))))


img_size = np.array(img.GetSize())
img_spacing = np.array(img.GetSpacing())

resample = sitk.ResampleImageFilter()
resample.SetInterpolator(sitk.sitkLinear if linear else sitk.sitkNearestNeighbor)

resample.SetSize(np.round(img_size * (img_spacing / np.array(spacing))).astype(int).tolist())
resample.SetOutputSpacing(spacing)
resample.SetOutputDirection(img.GetDirection())
resample.SetOutputOrigin(img.GetOrigin())

resam_img = resample.Execute(img)
arr = sitk.GetArrayFromImage(resam_img).transpose(2,1,0)


In [ ]:
print(f"vol.shape: {vol.shape}, resam_img.GetSize(): {resam_img.GetSize()}, arr.shape: {arr.shape}")


vol.shape: (167, 136, 41), resam_img.GetSize(): (114, 92, 82), arr.shape: (114, 92, 82)


In [ ]:
'''
vol.shape  ===> (167, 136, 41)
resam_img.shape ==> (114, 92, 82)
arr.shape ==> (114, 92, 82)
'''


(114, 92, 82)

In [ ]:
reshaped_arr = scipy.ndimage.zoom(
			arr,
			[t/s for t, s in zip(shape, arr.shape)], 	# Computes the scaling factor for each axis
			order=1 if linear else 0)                	# order=1: linear interpolation -> smoothing, for continuous data like CT intensities
													 	# order=0: nearest-neighbor interpolation -> no smoothing, for discrete labels/masks


In [ ]:
'''
If shape = (64, 64, 64) and arr.shape = (114, 92, 82)
factors = [64/114, 64/92, 64/82]
'''
interp = sitk.sitkLinear if linear else sitk.sitkNearestNeighbor

img_size = np.array(img.GetSize())
img_spacing = np.array(img.GetSpacing())

resample = sitk.ResampleImageFilter()
resample.SetInterpolator(sitk.sitkLinear if linear else sitk.sitkNearestNeighbor)

resample.SetSize(np.round(img_size * (img_spacing / np.array(spacing))).astype(int).tolist())
resample.SetOutputSpacing(spacing)
resample.SetOutputDirection(img.GetDirection())
resample.SetOutputOrigin(img.GetOrigin())

new_img = resample.Execute(img)
arr = sitk.GetArrayFromImage(new_img).transpose(2,1,0) #shape: (114, 92, 82)
arr = scipy.ndimage.zoom(
			arr,
			[t/s for t, s in zip(shape, arr.shape)], # Computes the scaling factor for each axis
			order=1 if linear else 0)


affine = np.eye(4)
affine[:3, :3] = np.array(new_img.GetDirection()).reshape(3,3) * np.array(new_img.GetSpacing())[:, None]
affine[:3, 3] = new_img.GetOrigin()
print(f"resampled orientation: {affine[:3, :3]}")


resampled shape: (114, 92, 82) - img.GetSize(): (114, 92, 82)
resampled orientation: [[-0.68000001  0.          0.        ]
 [ 0.          0.68000001  0.        ]
 [ 0.          0.          2.        ]]


In [ ]:
caseID="TTS_BLT_11022191_60F"
case = Case(caseID)
segs = case.resampledsegs
path = f"{case.casePath}/onehot_mask.nii.gz"
onhot = case.onehot_mask(segs, path)
print(f"onhot shape: {onhot.shape}, orientation: {onhot.orientation}")


In [ ]:
'''
[WARNING] Missing fullCT.nii.gz in CNTRL_KMC_92706_81F
[WARNING] Missing fullCT.nii.gz in TTS_CDA_38093134_74F
[WARNING] Missing fullCT.nii.gz in TTS_GA_39800503_22F
[WARNING] Missing fullCT.nii.gz in TTS_LW_12109724_62F
[WARNING] Missing fullCT.nii.gz in TTS_MA_6894703_78F
[WARNING] Missing fullCT.nii.gz in TTS_SSA_35792472_73F
'''
